# 🏥 MedSigLIP Fine-tuning for Nail Disease Classification on Kaggle

**Project**: Nail Disease Detection & Classification  
**Model**: Google's MedSigLIP (Medical SigLIP Vision-Language Model)  
**Platform**: Kaggle Notebooks  
**Dataset**: Custom nail disease images (7 categories - ZIP file)  
**Created**: January 2026  
**License**: Apache 2.0

---

## ✨ Key Features

- ✅ No Google Drive setup required
- ✅ Auto-detects ZIP files from `/kaggle/input`
- ✅ Automatic dataset extraction and validation
- ✅ GPU optimization (P100 available)
- ✅ Comprehensive error handling
- ✅ Real-time training visualization

---

## 📊 Expected Dataset Structure (in your ZIP)

```
nail-disease-dataset.zip
└── data/
    ├── train/                    (80% - ~5,300 images)
    │   ├── Acral_Lentiginous_Melanoma/
    │   ├── blue_finger/
    │   ├── clubbing/
    │   ├── Healthy_Nail/
    │   ├── Onychogryphosis/
    │   ├── pitting/
    │   └── psoriasis/
    └── test/                     (20% - ~1,350 images)
        ├── Acral_Lentiginous_Melanoma/
        ├── blue_finger/
        ├── clubbing/
        ├── Healthy_Nail/
        ├── Onychogryphosis/
        ├── pitting/
        └── psoriasis/
```

## 🎯 Nail Disease Categories

1. **Acral Lentiginous Melanoma (ALM)** - Black/brown lines under nail
2. **Blue Finger** - Blue discoloration of nail bed
3. **Clubbing** - Bulging, rounded nail appearance
4. **Healthy Nail** - Normal reference
5. **Onychogryphosis** - Thickened, curved nails
6. **Pitting** - Small depressions in nail plate
7. **Psoriasis** - Nail pitting and discoloration from psoriasis

---

## ✅ Expected Outcomes

- **Training Time**: 30-40 minutes (P100 GPU)
- **Expected Accuracy**: 88-95% on test set
- **Model Size**: ~420 MB (compressed)
- **Inference Time**: <500ms per image
- **Mobile Compatible**: Yes (TensorFlow Lite conversion included)
